### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from gspread_pandas import Spread, conf

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### CONSTANTES

### QUERIES

In [5]:
q_vouch = '''WITH orders_table AS (
    SELECT o.order_id AS id,
           o.order_status AS status,
           cn.country_name AS country
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    WHERE o.registered_date BETWEEN DATE('2021-05-01') AND DATE('2021-05-31'))
SELECT bt.id AS Id,
       bt.code AS Code,
       IFNULL(CAST(ot.id AS STRING),'-') AS Order_Id,
       IFNULL(CAST(ot.status AS STRING),'-') AS Order_Status,
       IFNULL(ot.country,'-') AS Country
FROM `peya-data-origins-pro.cl_barter.trades` AS bt
INNER JOIN orders_table AS ot ON bt.trade.order_id = ot.id'''

In [13]:
# Descargo la data
bq_vouch = pd.io.gbq.read_gbq(q_vouch, project_id='peya-argentina', dialect='standard')


Downloading: 100%|███████████████████████████████████████████████████████| 2288398/2288398 [05:44<00:00, 6638.05rows/s]


In [7]:
# Copio la data
vouch = bq_vouch.copy()

### TRABAJO

In [12]:
vouch.pivot_table(index=['Country','Order_Status'],values=['Order_Id'],aggfunc='size',fill_value=0).reset_index()

,Country,Order_Status,0
0,Argentina,CONFIRMED,1040656
1,Argentina,PENDING,247
2,Argentina,REJECTED,101693
3,Bolivia,CONFIRMED,54228
4,Bolivia,PENDING,2
5,Bolivia,REJECTED,5138
6,Chile,CONFIRMED,317134
7,Chile,PENDING,90
8,Chile,REJECTED,45210
9,Colombia,CONFIRMED,109567
